# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import os
import gmaps
import json
# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_data = pd.read_csv('../Output_data/cities.csv')
cities_data.dropna(subset=['Humidity (%)'],inplace = True) #drop missing values in Humidity
cities_data

,City,Cloudiness,Country,Date,Humidity (%),Lat,Long,Max Temp (F),Wind Speed (mph)
0,Semey,58,KZ,1586642334,68,50.41,80.23,38.30,4.81
1,Teguise,20,ES,1586642307,68,29.06,-13.56,66.20,12.75
2,Hobart,75,AU,1586642093,57,-42.88,147.33,52.00,13.87
3,Faya,44,SA,1586642335,55,18.39,42.45,66.20,1.81
4,Patnongon,0,PH,1586642336,69,10.91,121.99,78.58,4.97
...,...,...,...,...,...,...,...,...,...
554,Domoni,48,KM,1586642804,73,-12.26,44.53,83.37,6.78
555,Misratah,0,LY,1586642804,60,32.38,15.09,61.74,14.34
556,Mozarlândia,8,BR,1586642804,73,-14.74,-50.57,80.11,1.57
557,Gashua,100,NG,1586642805,13,12.87,11.05,83.82,8.97


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

locations = cities_data[['Lat', 'Long']].astype(float)
humidity = cities_data['Humidity (%)'].astype(float)
print(len(locations))
print(len(humidity))

559
559


In [4]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=4, opacity = 0.45)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# A max temperature lower than 80 degrees but higher than 70
# Wind speed less than 10 mph.
# Zero cloudiness.
# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
# Note: Feel free to adjust to your specifications but be sure to limit the number of rows returned    
#             by your API requests to a reasonable number.

vacation_df = cities_data[(cities_data['Max Temp (F)'] > 70) & (cities_data['Max Temp (F)'] < 80) & 
                       (cities_data['Wind Speed (mph)'] < 10) & (cities_data['Cloudiness'] == 0)]


hotel_df = pd.DataFrame(vacation_df)
hotel_df["Hotel"] = ""
hotel_df 

,City,Cloudiness,Country,Date,Humidity (%),Lat,Long,Max Temp (F),Wind Speed (mph),Hotel
4,Patnongon,0,PH,1586642336,69,10.91,121.99,78.58,4.97,
73,Salto,0,UY,1586642218,50,-31.38,-57.97,75.20,0.98,
99,Mossamedes,0,AO,1586642280,61,-15.20,12.15,77.59,3.62,
101,Walvis Bay,0,NaN,1586642364,73,-22.96,14.51,73.00,7.90,
242,Colina,0,BR,1586642139,59,-20.71,-48.54,70.02,4.12,
381,Tura,0,IN,1586642462,32,25.52,90.22,72.18,4.45,
387,Goya,0,AR,1586642467,47,-29.14,-59.26,73.44,5.73,
404,Pisco,0,PE,1586642219,30,-13.70,-76.22,75.00,5.82,
449,Byron Bay,0,AU,1586642484,87,-28.65,153.62,72.00,2.24,
450,São Borja,0,BR,1586642485,53,-28.66,-56.00,70.75,6.62,


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [10]:
target_radius = 5000
target_type = "lodging" 

# set up a parameters dictionary
params = {
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Long"]
    city = row["City"]
    
     # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    print(f"Retrieving Results for nearest hotels/lodging in {city}.")
    response = requests.get(base_url, params=params).json()
   
    
    #print(json.dumps(response, indent=4, sort_keys=True))

    # extract results
    results =  response['results']
          
    
    try:
        print(f"The closest hotel/lodging is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("There are no hotels/lodging.")
    print("----------------------------------------------")

Retrieving Results for nearest hotels/lodging in Patnongon.
The closest hotel/lodging is La Veneranda Guest House.
----------------------------------------------
Retrieving Results for nearest hotels/lodging in Salto.
The closest hotel/lodging is Gran Hotel Uruguay.
----------------------------------------------
Retrieving Results for nearest hotels/lodging in Mossamedes.
The closest hotel/lodging is Chik-Chik Namibe.
----------------------------------------------
Retrieving Results for nearest hotels/lodging in Walvis Bay.
The closest hotel/lodging is Oyster Box Guesthouse.
----------------------------------------------
Retrieving Results for nearest hotels/lodging in Colina.
The closest hotel/lodging is Hotel e Pousada Dona Lidia.
----------------------------------------------
Retrieving Results for nearest hotels/lodging in Tura.
The closest hotel/lodging is Hotel RIKMAN Continental.
----------------------------------------------
Retrieving Results for nearest hotels/lodging in Goya

In [11]:
# hotel_df
# hotel_df.dropna(subset = ["Hotel"],inplace= True)
hotel_df = hotel_df[hotel_df['Hotel'] != '']
hotel_df

,City,Cloudiness,Country,Date,Humidity (%),Lat,Long,Max Temp (F),Wind Speed (mph),Hotel
4,Patnongon,0,PH,1586642336,69,10.91,121.99,78.58,4.97,La Veneranda Guest House
73,Salto,0,UY,1586642218,50,-31.38,-57.97,75.20,0.98,Gran Hotel Uruguay
99,Mossamedes,0,AO,1586642280,61,-15.20,12.15,77.59,3.62,Chik-Chik Namibe
101,Walvis Bay,0,NaN,1586642364,73,-22.96,14.51,73.00,7.90,Oyster Box Guesthouse
242,Colina,0,BR,1586642139,59,-20.71,-48.54,70.02,4.12,Hotel e Pousada Dona Lidia
381,Tura,0,IN,1586642462,32,25.52,90.22,72.18,4.45,Hotel RIKMAN Continental
387,Goya,0,AR,1586642467,47,-29.14,-59.26,73.44,5.73,Apart Costanera
404,Pisco,0,PE,1586642219,30,-13.70,-76.22,75.00,5.82,Hostal San Isidro Oficial
449,Byron Bay,0,AU,1586642484,87,-28.65,153.62,72.00,2.24,Backpackers Inn on the Beach
450,São Borja,0,BR,1586642485,53,-28.66,-56.00,70.75,6.62,Hotel Executivo


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Long"]]


In [13]:

# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info, display_info_box=True)

fig.add_layer(markers)
fig


Figure(layout=FigureLayout(height='420px'))